Author : Rohit Kumar Hansdah
Last Updated : 23 December

In [ ]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.utils import shuffle
import lightgbm as lgb
from lightgbm import LGBMClassifier
import eli5

from catboost import CatBoostClassifier
import xgboost as xgb




import riiideducation

%matplotlib inline
# for heatmap and other plots
colorMap1 = sns.color_palette("RdBu_r")
# for countplot and others plots
colorMap2 = 'Blues_r'



In [ ]:
sampler = TPESampler(
    seed=666
)

In [ ]:
types = {
        'row_id': 'int64', 
        'timestamp': 'int64', 
        'user_id': 'int32', 
        'content_id': 'int16', 
        'content_type_id': 'int8',
        'task_container_id': 'int16', 
        'user_answer': 'int8', 
        'answered_correctly': 'int8', 
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'boolean'
}

In [ ]:
train = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv', 
    low_memory=False, 
    nrows=10**6, 
    dtype=types
)
questions=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
test=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')

train.head()

#                  DATA EXPLORATION & EDA

**TRAIN**

**row_id**: (int64) ID code for the row.

**timestamp**: (int64) the time in milliseconds between this user interaction and the first event completion from that user.

**user_id**: (int32) ID code for the user.

**content_id**: (int16) ID code for the user interaction

**content_type_id**: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

**task_container_id**: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

**user_answer**: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

**answered_correctly**: (int8) if the user responded correctly. Read -1 as null, for lectures.

**prior_question_elapsed_time**: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures
in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

**prior_question_had_explanation**: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.

In [ ]:
print(f"Train shape: {train.shape}")

In [ ]:

train.head()

In [ ]:
train.describe().style.background_gradient(cmap='Blues')

In [ ]:
print(f'Number of unique users: {len(np.unique(train.user_id))}')


Lets look at the missing data

In [ ]:
print(train.isnull().sum()/len(train))

Also let's check a correlation matrix to get more information between the columns

In [ ]:
corr_matrix=train.corr()
corr_matrix['answered_correctly'].sort_values(ascending=True)

In [ ]:
plt.figure(figsize=(13,10))
sns.heatmap(corr_matrix,annot=True,
           linewidths=5,cmap=colorMap1)







Let's check the distribution of **prior_question_elapsed_time** 

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.countplot(x="prior_question_elapsed_time", 
                   data=train[train['prior_question_elapsed_time'].notnull()],
                   palette=colorMap2)


Let's check any connection between our target value and a frequency of answering questions

In [ ]:
freq_answered_tasks = train['task_container_id'].value_counts().reset_index()
freq_answered_tasks.columns = [
    'task_container_id', 
    'freq'
]
train['freq_task_id'] = ''
train.loc[train['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] < 10000]['task_container_id'].values), 'freq_task_id'] = 'very rare answered'
train.loc[train['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] >= 10000]['task_container_id'].values), 'freq_task_id'] = 'rare answered'
train.loc[train['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] >= 50000]['task_container_id'].values), 'freq_task_id'] = 'normal answered'
train.loc[train['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] >= 200000]['task_container_id'].values), 'freq_task_id'] = 'often answered'
train.loc[train['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] >= 400000]['task_container_id'].values), 'freq_task_id'] = 'very often answered'



In [ ]:
train.sample(5)

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(x='freq_task_id', hue='answered_correctly', data=train, palette=colorMap2)

In [ ]:
WIDTH=800

In [ ]:
ds = train['answered_correctly'].value_counts().reset_index()

ds.columns = [
    'answered_correctly', 
    'percent_of_answers'
]

ds['percent_of_answers'] /= len(train)
ds = ds.sort_values(['percent_of_answers'])

fig = px.pie(
    ds, 
    names='answered_correctly', 
    values='percent_of_answers', 
    title='Percent of correct answers', 
    width=WIDTH,
    height=500 
)


fig.show()

prior_question_had_explanation has a medium correlation with the target value. So let's look at his distribution



In [ ]:
plt.figure(figsize=(15, 11))
ax = sns.countplot(x="prior_question_had_explanation", hue="answered_correctly", data=train[train['prior_question_had_explanation'].notnull()], palette=colorMap2)

The answer's showing increases the probability of a successful answering. Let's go further.
Check the most active user_id



In [ ]:
N=30

user_freq = train['user_id'].value_counts().reset_index()
user_freq.columns = [
    'user_id', 
    'count'
]

# Add ' - ' to convert user_id to str and not sort
user_freq['user_id'] = user_freq['user_id'].astype(str) + ' - '
user_freq = user_freq.sort_values(['count'], ascending=False).head(N)

plt.figure(figsize=(15, 15))
sns.barplot(x='count', y='user_id', data=user_freq, orient='h', palette=colorMap2)
plt.title(f'Top {N} the most active users', fontsize=14)

And the most useful content_id

In [ ]:
WIDTH=800

Also we need to check distribution in content_type_id: number of video lectures and questions

In [ ]:
ds = train['content_type_id'].value_counts().reset_index()

ds.columns = [
    'content_type_id', 
    'percent'
]

ds['percent'] /=len(train)

fig = px.pie(
    ds, 
    names='content_type_id', 
    values='percent', 
    title='Lecures & questions', 
    width=WIDTH,
    height=500 
)

fig.show()

In [ ]:
ds=train['user_answer'].value_counts().reset_index()
ds.columns = [
    'user_answer', 
    'percent_of_answers'
]
ds['percent_of_answers']/=len(train)
ds = ds.sort_values(['percent_of_answers'])
fig = px.bar(
    ds, 
    x='user_answer', 
    y='percent_of_answers', 
    orientation='v', 
    title='Percent of user answers for every option', 
    width=WIDTH,
    height=400 
)

fig.show()

In [ ]:
task_ids_freq = train['task_container_id'].value_counts().reset_index()
task_ids_freq.columns = ['task_container_id', 'count']

fig, ax = plt.subplots(figsize=(15, 10))

sns.pointplot(x='task_container_id', y='count', data=task_ids_freq, palette=colorMap2)
xticks_range = range(min(task_ids_freq['task_container_id']), 
                     max(task_ids_freq['task_container_id']),
                     1000)
plt.xticks(list(xticks_range), list(xticks_range))

**QUESTIONS.CSV**


question_id: foreign key for the train/test content_id column, when the content type is question (0).

bundle_id: code for which questions are served together.

correct_answer: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

part: top level category code for the question.

tags: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

In [ ]:
questions.head()

In [ ]:
questions.describe().style.background_gradient(cmap='Blues')

In [ ]:
print(questions.isnull().sum() / len(questions))

In [ ]:





questions['tag'] = questions['tags'].str.split(' ')
questions = questions.explode('tag')
questions = pd.merge(
    questions, 
    questions.groupby('question_id')['tag'].count().reset_index(), 
    on='question_id'
    )

questions = questions.drop(['tag_x'], axis=1)

questions.columns = [
    'question_id', 
    'bundle_id', 
    'correct_answer', 
    'part', 
    'tags', 
    'tags_number'
]
questions = questions.drop_duplicates()

In [ ]:
ds = questions['correct_answer'].value_counts().reset_index()

ds.columns = [
    'correct_answer', 
    'number_of_answers'
]

ds['correct_answer'] = ds['correct_answer'].astype(str) + '-'
ds = ds.sort_values(['number_of_answers'])

fig = px.bar(
    ds, 
    x='number_of_answers', 
    y='correct_answer', 
    orientation='h', 
    title='Number of correct answers per group', 
    width=WIDTH,
    height=300
)
fig.show()

In [ ]:
ds = questions['part'].value_counts().reset_index()

ds.columns = [
    'part', 
    'count'
]

ds['part'] = ds['part'].astype(str) + '-'
ds = ds.sort_values(['count'])

fig = px.bar(
    ds, 
    x='count', 
    y='part', 
    orientation='h', 
    title='Parts distribution',
    width=WIDTH,
    height=400
)
fig.show()

In [ ]:
ds = questions['tags_number'].value_counts().reset_index()

ds.columns = [
    'tags_number', 
    'count'
]

ds['tags_number'] = ds['tags_number'].astype(str) + '-'
ds = ds.sort_values(['tags_number'])

fig = px.bar(
    ds, 
    x='count', 
    y='tags_number', 
    orientation='h', 
    title='Number tags distribution', 
    width=WIDTH,
    height=400 
    )

fig.show()

In [ ]:


check = questions['tags'].str.split(' ').explode('tags').reset_index()
check = check['tags'].value_counts().reset_index()

check.columns = [
    'tag', 
    'count'
]

check['tag'] = check['tag'].astype(str) + '-'
check = check.sort_values(['count']).tail(40)

fig = px.bar(
    check, 
    x='count', 
    y='tag', 
    orientation='h', 
     title='Top 40 most useful tags', 
    width=WIDTH,
    height=900 
)

fig.show()


LECTURES.CSV

Metadata for the lectures watched by users as they progress in their education.

**lecture_id**: foreign key for the train/test content_id column, when the content type is lecture (1).

**part**: top level category code for the lecture.

**tag**: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

**type_of**: brief description of the core purpose of the lecture

In [ ]:
lectures.head(10)

In [ ]:
print('Part of missing values for every column')
print(lectures.isnull().sum() / len(lectures))

In [ ]:
lectures['type_of'].value_counts()

**Test.csv**

In [ ]:
test.head()

# FEATURE ENGINEERING

In [ ]:
train.head()

In [ ]:
features_df = train.iloc[:int(9/10 * len(train))]
train = train.iloc[int(9/10 * len(train)):]

In [ ]:
train_questions_only_df = features_df[features_df['answered_correctly']!=-1]
grouped_by_user_df = train_questions_only_df.groupby('user_id')

user_answers_df = grouped_by_user_df.agg(
    {
        'answered_correctly': [
            'mean', 
            'count', 
            'std', 
            'median', 
            'skew'
        ]
    }
).copy()

user_answers_df.columns = [
    'mean_user_accuracy',
    'questions_answered',
    'std_user_accuracy', 
    'median_user_accuracy', 
    'skew_user_accuracy'
]

In [ ]:
grouped_by_content_df = train_questions_only_df.groupby('content_id')
content_answers_df = grouped_by_content_df.agg(
    {
        'answered_correctly': [
            'mean', 
            'count', 
            'std', 
            'median', 
            'skew'
        ]
    }
).copy()

content_answers_df.columns = [
    'mean_accuracy', 
    'question_asked', 
    'std_accuracy', 
    'median_accuracy', 
    'skew_accuracy'
]
content_answers_df

In [ ]:
del features_df
del grouped_by_user_df
del grouped_by_content_df

In [ ]:
features = [
    'mean_user_accuracy', 
    'questions_answered',
    'std_user_accuracy', 
    'median_user_accuracy',
    'skew_user_accuracy',
    'mean_accuracy', 
    'question_asked',
    'std_accuracy', 
    'median_accuracy',
    'prior_question_elapsed_time', 
    'prior_question_had_explanation',
    'skew_accuracy'
]

target = 'answered_correctly'

In [ ]:
target

In [ ]:
train = train[train[target] != -1]

In [ ]:
train = train.merge(user_answers_df, how='left', on='user_id')
train = train.merge(content_answers_df, how='left', on='content_id')

In [ ]:
target

In [ ]:

train['prior_question_had_explanation'] = train['prior_question_had_explanation'].fillna(value=False).astype(bool)
df = train.fillna(value=0.5)

In [ ]:
col_to_drop = set(train.columns.values.tolist()).difference(features + [target])
for col in col_to_drop:
    del df[col]

In [ ]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0.5)

# Modeling

In [ ]:
train_df, test_df, y_train, y_test = train_test_split(df[features], df[target], random_state=777, test_size=0.2)

# LGBM classifier

In [ ]:
params = {
    'num_leaves': 30, 
    'n_estimators': 300, 
    'min_data_in_leaf': 100, 
    'max_depth': 5, 
    'lambda': 0.0, 
    'feature_fraction': 1.0
}


In [ ]:
model = LGBMClassifier(**params)
model.fit(train_df, y_train)

In [ ]:
print('LGB ROC-AUC score: ', roc_auc_score(y_test.values, model.predict_proba(test_df)[:, 1]))

# Random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier()
rfclf.fit(train_df,y_train)
pred = rfclf.predict(test_df)

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test,pred))

# Xgboost classifier

In [ ]:
from xgboost import XGBClassifier
xgbclf = XGBClassifier()
xgbclf.fit(train_df,y_train)
xgb_pred = xgbclf.predict(test_df)

In [ ]:
print(roc_auc_score(y_test,xgb_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(train_df,y_train)
mlp_pred = mlp.predict(test_df)

In [ ]:
print(roc_auc_score(y_test,mlp_pred))

In [ ]:
eli5.show_weights(model, top=20)

In [ ]:
lgb.plot_importance(model)

Submission preparation

In [ ]:
env = riiideducation.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    # merge
    test_df = test_df.merge(user_answers_df, on = "user_id", how = "left")
    #test_df = test_df.merge(task_container_characteristics, on = "task_container_id", how = "left")
    test_df = test_df.merge(content_answers_df, on = "content_id", how = "left")
    
    # type transformation
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value=False).astype(bool)
    test_df.fillna(value = 0.5, inplace = True)
    test_df = test_df.replace([np.inf, -np.inf], np.nan)
    test_df = test_df.fillna(0.5)
    
    # preds
    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:, 1]
    cols_to_submission = ['row_id', 'answered_correctly', 'group_num']
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])